# Artificial Neural Network

In [391]:
import random
import copy
import numpy as np

In [392]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        #parameters for Adam optimizer
        self.vdb = [np.zeros(b.shape) for b in self.biases]
        self.vdw = [np.zeros(w.shape) for w in self.weights]
        self.sdb = [np.zeros(b.shape) for b in self.biases]
        self.sdw = [np.zeros(w.shape) for w in self.weights]
        
        
        
    def feedforward(self, a,using_soft_max=False):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases[:-1], self.weights[:-1]):
            a = sigmoid(np.dot(w, a)+ b)
            
        b,w = (self.biases[-1],self.weights[-1])
        if using_soft_max:
            a = soft_max(np.dot(w, a)+ b)
        else:
            a = sigmoid(np.dot(w, a)+ b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta,test_data=None,using_soft_max = False,using_adam_optimizer = False):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data != None:
            n_test = len(test_data)

        n = len(training_data)
        fail=0
        maximum=0
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, j, using_soft_max=using_soft_max,using_adam_optimizer=using_adam_optimizer)
            if test_data != None:
                
                actual=self.evaluate(test_data,using_soft_max)
                #print("actual {0} max {1}".format(actual,maximum))
                if (actual/n_test)>maximum:
                    maximum=actual/n_test
                    fail=0
                    network=copy.deepcopy(self)
                else:
                    fail+=1
                print("Epoch {0}: {1} / {2}, {3}%".format(j, actual, n_test,actual/n_test))
                print(fail)
                if fail==20:
                    break
            else:
                print("Epoch {0} complete".format(j))
        network.saveWeightsAndBiases("Network2")
    
    def update_mini_batch(self, mini_batch, eta,epoch,using_soft_max=False,using_adam_optimizer = False):
        beta1 = 0.9 
        beta2 = 0.999
        epsilon = np.array([pow(10, -8)])
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y, soft_max)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        #Adam optimizer
        if using_adam_optimizer:
            self.vdw = np.array([beta1*vv+(1-beta1)*nw for vv, nw in zip(self.vdw,nabla_w)])
            self.vdb = np.array([beta1*bb+(1-beta1)*nb for bb, nb in zip(self.vdb,nabla_b)])
            self.sdw = np.array([beta2*ss+(1-beta2)*pow(nw, 2) for ss, nw in zip(self.sdw,nabla_w)])
            self.sdb = np.array([beta2*ss+(1-beta2)*pow(nw, 2) for ss, nw in zip(self.sdb,nabla_b)])
            vdw_corrected = self.vdw / (1-pow(beta1, epoch+1))
            vdb_corrected = self.vdb / (1-pow(beta1, epoch+1))
            sdw_corrected = self.sdw / (1-pow(beta2,epoch+1))
            sdb_corrected = self.sdb / (1-pow(beta2,epoch+1))
            self.weights = [w - (eta * (v_corrected / (np.sqrt(s_corrected)+ epsilon)))
                        for w, v_corrected, s_corrected in zip(self.weights,vdw_corrected,sdw_corrected)]
            self.biases = [b - (eta * (v_corrected / (np.sqrt(s_corrected)+ epsilon)))
                        for b, v_corrected, s_corrected in zip(self.biases,vdb_corrected,sdb_corrected)]
        else:     
            self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
            self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]
    
    def backprop(self, x, y, using_soft_max=False,using_cross_entropy = True):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases[:-1], self.weights[:-1]):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        w,b = (self.weights[-1],self.biases[-1])
        z = np.dot(w, activation)+b
        zs.append(z)
        if using_soft_max:
            activation = soft_max(z)
        else:
            activation = sigmoid(z)
        activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * soft_max_prime(zs[-1])
        if using_cross_entropy:
            delta = self.cost_derivative(activations[-1], y)
        else:
            delta = self.cost_derivative(activations[-1], y) * soft_max_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data, soft_max):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x,using_soft_max=soft_max)), np.argmax(y)) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)
    
    def saveWeightsAndBiases(self,fileName):
        np.savez(fileName,self.weights,self.biases)
    
    def loadWeightsAndBiases(self,fileName):
        npzfile = np.load(fileName)
        self.weights = npzfile['arr_0']
        self.biases = npzfile['arr_1']
        
    
    

### Activation Functions

In [393]:
def soft_max(z):
    expA = np.exp(z - np.max(z))
    res = expA / expA.sum()
    return res

In [394]:
def soft_max_prime(z):
    return soft_max(z)*(1-soft_max(z))

In [395]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

In [396]:
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
def reluDerivative(x):
    x[x<=0] = 0
    return x

In [ ]:
def reluprime(x):
    return np.where(x > 0, 1.0, 0.0)

## Preparing data

In [397]:
%cd C:\Users\marti\Desktop\Universidad\Suecia\Int sys\DE

C:\Users\marti\Desktop\Universidad\Suecia\Int sys\DE


In [398]:
import pandas as pd
import csv
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [399]:
supported_genres=['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']

In [400]:
#normalising all features
data = pd.read_csv('predictions_cnn_19k.csv')
genre_list=[]

for row in  data.loc[:,'filename']:
    genre_list.append(row.split('.')[0])

minMaxScaler = MinMaxScaler()
X=minMaxScaler.fit_transform(np.array(data.iloc[:,3:-10], dtype = float))

normalised_values_table = pd.concat([data.loc[:,['filename','offset','duration']],
                           pd.DataFrame(X,columns=data.columns[3:-10]),
                           data.loc[:,supported_genres],
                           pd.DataFrame(np.array(genre_list),columns=['genre'])],
                           axis=1)
normalised_values_table.to_csv('Normalised_Features.csv',index=False)

In [401]:
#calculating desired output according to genre column in table
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
auxArray = np.array([[1],[0],[0],[0],[0],[0],[0],[0],[0],[0]])
desiredOutput = [np.roll(auxArray,x) for x in y]
desiredOutput = np.array(desiredOutput)

In [402]:
#in this part we include which features we want to give to ANN and combine them with predictions
USING_PREDICTIONS = True
data = pd.read_csv('Normalised_Features.csv')
features_selection = ['chroma_stft','spectral_centroid','spectral_bandwidth','rolloff','zero_crossing_rate', 'mfcc']
new_feature_table = pd.DataFrame()

for feature in features_selection:
    if feature != 'mfcc':
        new_feature_table = pd.concat([new_feature_table, data.loc[:,[feature]]],axis=1)
    else:
        new_feature_table = pd.concat([new_feature_table, data.filter(regex=("mfcc.*"))],axis=1)

if USING_PREDICTIONS:       
    features_and_predictions = pd.concat([new_feature_table,data.loc[:,supported_genres]],axis=1)
else:
    features_and_predictions = new_feature_table   

ANN_NUM_OF_INPUT = len(features_and_predictions.columns)
ANN_NUM_OF_OUTPUT = len(supported_genres)
#features_and_predictions[:10]


In [403]:
data_set=[]

for row_index,row in features_and_predictions.iterrows():
    r = np.array(row)
    data_set.append(np.reshape(r,(len(row),1)))
data_set = [(x,y) for x,y in zip(data_set,desiredOutput)]
#random.shuffle(data_set)  

In [404]:
LEARNING_DATASET_P = 0.5
EVALUATION_DATASET_P = 0.2
TEST_DATASET_P = 0.3

l = len(data_set)
frames_per_song = 19 #we have 4 samples for 1 song
num_of_songs = int(l/frames_per_song)
songs = np.array(data_set).reshape(num_of_songs,frames_per_song,2)
#np.random.shuffle(songs)

i = int(num_of_songs*LEARNING_DATASET_P)
j = int(num_of_songs*EVALUATION_DATASET_P)
t = int(num_of_songs*TEST_DATASET_P)

test_dataset= np.reshape(songs[:t],(t*frames_per_song,2))
learning_dataset = np.reshape(songs[t:t+i],(i*frames_per_song,2))
evaluation_dataset = np.reshape(songs[t+i:t+i+j],(j*frames_per_song,2))

np.random.shuffle(learning_dataset)
np.random.shuffle(evaluation_dataset)
np.random.shuffle(test_dataset)

## Learning

In [405]:
ann = Network([ANN_NUM_OF_INPUT,20,15,ANN_NUM_OF_OUTPUT])

In [406]:
ann.SGD(learning_dataset.tolist(),500,32,0.01,test_data=evaluation_dataset.tolist(), using_soft_max=True,using_adam_optimizer=True)

Epoch 0: 3237 / 3800, 0.8518421052631578%
0
Epoch 1: 3248 / 3800, 0.8547368421052631%
0
Epoch 2: 3262 / 3800, 0.858421052631579%
0
Epoch 3: 3269 / 3800, 0.8602631578947368%
0
Epoch 4: 3274 / 3800, 0.861578947368421%
0
Epoch 5: 3266 / 3800, 0.8594736842105263%
1
Epoch 6: 3265 / 3800, 0.8592105263157894%
2
Epoch 7: 3257 / 3800, 0.8571052631578947%
3
Epoch 8: 3264 / 3800, 0.8589473684210527%
4
Epoch 9: 3265 / 3800, 0.8592105263157894%
5


In [407]:
ann.loadWeightsAndBiases("Network2.npz")

In [411]:
print(ann.evaluate(test_data=test_dataset.tolist(),soft_max=True)/len(evaluation_dataset.tolist()))

0.9763157894736842
